In [ ]:
##Question1: Create a single dataframe with the concatenation of all input csv files, adding a column called country

In [77]:
import pandas as pd   
import glob        
import os      

csv_files = glob.glob('*videos.csv')     
csv_files

['MXvideos.csv',
 'INvideos.csv',
 'DEvideos.csv',
 'JPvideos.csv',
 'KRvideos.csv',
 'CAvideos.csv',
 'RUvideos.csv',
 'FRvideos.csv',
 'USvideos.csv',
 'GBvideos.csv']

In [92]:
df_list = []

for file in csv_files:
    filename = os.path.basename(file)
    country_code = filename[:2]

    try:
        temp_df = pd.read_csv(file, encoding='utf-8-sig')
    except UnicodeDecodeError:
        temp_df = pd.read_csv(file, encoding='latin1')

    temp_df['country'] = country_code
    df_list.append(temp_df)

combined_df = pd.concat(df_list, ignore_index=True)

In [ ]:
##Question2: Extract all videos that have no tag

In [85]:
combined_df['tags']

0         MasterChef Junior 2017|"TV Azteca"|"recetas"|"...
1                                           La Voz Mexico 7
2         Chumel Torres|"El Pulso de la Republica"|"noti...
3                temblor|"costa rica"|"sismo en costa rica"
4         MUSAS|"lesslie"|"karen"|"hacks"|"perros"|"dogs...
                                ...                        
375937    Enrique Iglesias feat. Pitbull|"MOVE TO MIAMI"...
375938    jacob sartorius|"jacob"|"up with it"|"jacob sa...
375939    anne|"marie"|"anne-marie"|"2002"|"two thousand...
375940    Eurovision Song Contest|"2018"|"Lisbon"|"Cypru...
375941    Kyle|"SuperDuperKyle"|"Ikuyo"|"2 Chainz"|"Soph...
Name: tags, Length: 375942, dtype: object

In [86]:
no_tag_mask = (
    (combined_df['tags'] == '[none]') |     
    (combined_df['tags'].isna()) |           
    (combined_df['tags'].str.strip() == '')     
)

In [87]:
no_tag_videos = combined_df[no_tag_mask].copy()   

In [11]:
##Question3: For each channel, determine the total number of views

In [89]:
combined_df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description', 'country'],
      dtype='object')

In [90]:
channel_views = (
    combined_df
    .groupby('channel_title', as_index=False)['views']
    .sum() )

In [95]:
channel_views_sorted = channel_views.sort_values('views', ascending=False)
channel_views_sorted.head(10)

,channel_title,views
4590,ChildishGambinoVEVO,11016766510
15599,Marvel Entertainment,10430605449
17793,NickyJamTV,9479859505
18533,Ozuna,8623329509
28519,ibighit,8205572221
6724,DrakeVEVO,7637228580
2801,Bad Bunny,7124207494
2110,ArianaGrandeVevo,6202230488
28728,jypentertainment,5802822913
7084,Ed Sheeran,5775405574


In [16]:
##Question4: Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.

In [33]:
mask_excluded = (
    (combined_df['comments_disabled'] & combined_df['ratings_disabled']) |
    (combined_df['video_error_or_removed'])
)

In [34]:
mask_excluded.sum()

np.int64(2620)

In [35]:
excluded = combined_df[mask_excluded].copy()

In [38]:

combined_df = combined_df[~mask_excluded].copy()

combined_df.reset_index(drop=True, inplace=True)

In [23]:
##Question5:Add a like_ratio column storing the ratio between the number of likes and of dislikes

In [40]:
import numpy as np

In [41]:
combined_df['like_ratio'] = np.where(
    combined_df['dislikes'] > 0,
    combined_df['likes'] / combined_df['dislikes'],np.nan)

In [42]:
combined_df[['likes', 'dislikes', 'like_ratio']].head(10)

,likes,dislikes,like_ratio
0,4182,361,11.584488
1,271,174,1.557471
2,10105,266,37.988722
3,378,171,2.210526
4,57781,681,84.847283
5,506,67,7.552239
6,2277,69,33.000000
7,7745,659,11.752656
8,20155,912,22.099781
9,83582,2194,38.095716


In [28]:
##Question6: Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)

In [44]:
combined_df['publish_time'].dtype

dtype('O')

In [45]:
combined_df['publish_time'] = pd.to_datetime(combined_df['publish_time'], errors='coerce')

In [46]:
combined_df['publish_time'].dtype

datetime64[ns, UTC]

In [47]:
combined_df['publish_time_10min'] = combined_df['publish_time'].dt.floor('10min')
combined_df[['publish_time', 'publish_time_10min']].head(10)

,publish_time,publish_time_10min
0,2017-11-13 06:06:22+00:00,2017-11-13 06:00:00+00:00
1,2017-11-13 05:11:58+00:00,2017-11-13 05:10:00+00:00
2,2017-11-13 17:00:02+00:00,2017-11-13 17:00:00+00:00
3,2017-11-13 03:47:10+00:00,2017-11-13 03:40:00+00:00
4,2017-11-13 19:17:48+00:00,2017-11-13 19:10:00+00:00
5,2017-11-12 20:17:38+00:00,2017-11-12 20:10:00+00:00
6,2017-11-13 07:00:03+00:00,2017-11-13 07:00:00+00:00
7,2017-11-13 06:18:23+00:00,2017-11-13 06:10:00+00:00
8,2017-11-12 19:54:12+00:00,2017-11-12 19:50:00+00:00
9,2017-11-12 21:14:44+00:00,2017-11-12 21:10:00+00:00


In [35]:
##Question7:For each interval, determine the number of videos, average number of likes and of dislikes

In [48]:
interval_stats = (
    combined_df
    .groupby('publish_time_10min')
    .agg(
        videos_count=('video_id', 'count'),   
        avg_likes=('likes', 'mean'),          
        avg_dislikes=('dislikes', 'mean'))       
    .reset_index()  
)
interval_stats.head(10)

,publish_time_10min,videos_count,avg_likes,avg_dislikes
0,2006-07-23 08:20:00+00:00,1,459.000000,152.000000
1,2007-03-05 16:20:00+00:00,9,336.666667,2.000000
2,2007-06-25 06:50:00+00:00,12,579.833333,11.500000
3,2007-12-03 20:50:00+00:00,16,187.937500,15.687500
4,2008-01-07 21:20:00+00:00,10,99.900000,2.000000
5,2008-01-13 01:30:00+00:00,2,1417.000000,49.500000
6,2008-02-12 20:20:00+00:00,3,1985.666667,124.666667
7,2008-04-05 18:20:00+00:00,4,46.000000,6.000000
8,2008-06-17 00:00:00+00:00,4,469.000000,4.000000
9,2008-08-07 12:10:00+00:00,3,78.333333,1.000000


In [37]:
##Question8: For each tag, determine the number of videos. Notice that tags contains a string with several tags.

In [53]:
tags_df = combined_df.copy()
tags_df = tags_df[
    tags_df['tags'].notna() &         
    (tags_df['tags'] != '[none]') &    
    (tags_df['tags'].str.strip() != '') ].copy()

In [54]:
tags_df[['video_id', 'tags']].head()

,video_id,tags
0,SbOwzAl9ZfQ,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""..."
1,klOV6Xh-DnI,La Voz Mexico 7
2,6L2ZF7Qzsbk,"Chumel Torres|""El Pulso de la Republica""|""noti..."
3,hcY52MFWMDM,"temblor|""costa rica""|""sismo en costa rica"""
4,_OXDcGPVAa4,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs..."


In [59]:
tags_df['tag_list'] = tags_df['tags'].str.split('|')  
tags_df[['video_id', 'tag_list']].head()

,video_id,tag_list
0,SbOwzAl9ZfQ,"[MasterChef Junior 2017, ""TV Azteca"", ""recetas..."
1,klOV6Xh-DnI,[La Voz Mexico 7]
2,6L2ZF7Qzsbk,"[Chumel Torres, ""El Pulso de la Republica"", ""n..."
3,hcY52MFWMDM,"[temblor, ""costa rica"", ""sismo en costa rica""]"
4,_OXDcGPVAa4,"[MUSAS, ""lesslie"", ""karen"", ""hacks"", ""perros"",..."


In [60]:
tags_exploded = tags_df.explode('tag_list').copy()
tags_exploded[['video_id', 'tag_list']].head()

,video_id,tag_list
0,SbOwzAl9ZfQ,MasterChef Junior 2017
0,SbOwzAl9ZfQ,"""TV Azteca"""
0,SbOwzAl9ZfQ,"""recetas"""
0,SbOwzAl9ZfQ,"""cocina"""
0,SbOwzAl9ZfQ,"""Anette Michel"""


In [94]:
tag_counts = (
    tags_exploded
    .groupby('tag_list')['video_id']
    .nunique()   
    .reset_index(name='videos_count')
    .sort_values('videos_count', ascending=False)
)

In [116]:
##Question9:Find the tags with the largest number of videos

In [44]:
top_10_tags = tag_counts.head(10)
top_10_tags

,tag_list,videos_count
12773,"""2018""",5563
343032,"""funny""",4096
283298,"""comedy""",3044
452295,"""news""",2794
11973,"""2017""",2576
520374,"""show""",2034
571106,"""video""",2014
544833,"""television""",1716
561261,"""tv""",1700
444304,"""music""",1465


In [45]:
##Question10:For each (tag, country) pair, compute average ratio likes/dislikes

In [103]:
tag_country_ratio = (
    tags_exploded
    .groupby(['tag_list', 'country'])
    .agg(
        avg_like_ratio=('like_ratio', 'mean'),
        n_videos=('video_id', 'nunique')         
    )
    .reset_index()
)

In [ ]:
##Question11: For each (trending_date, country) pair, the video with the largest number of views

In [54]:
idx = combined_df.groupby(['trending_date', 'country'])['views'].idxmax()
idx.head()

trending_date  country
17.01.12       CA         175534
               DE          80652
               FR         257054
               GB         338276
               IN          43515
Name: views, dtype: int64

In [56]:
##Question12: Divide trending_date into three columns: year, month, day

In [57]:
combined_df['trending_date'].head(10)

0    17.14.11
1    17.14.11
2    17.14.11
3    17.14.11
4    17.14.11
5    17.14.11
6    17.14.11
7    17.14.11
8    17.14.11
9    17.14.11
Name: trending_date, dtype: object

In [58]:
combined_df['trending_date'].dtype

dtype('O')

In [117]:
combined_df['trending_date'] = pd.to_datetime(
    combined_df['trending_date'],
    format='%y.%d.%m',  
    errors='coerce' 
)

In [60]:
combined_df['trending_date'].dtype

dtype('<M8[ns]')

In [61]:
combined_df['trending_date'].head()

0   2017-11-14
1   2017-11-14
2   2017-11-14
3   2017-11-14
4   2017-11-14
Name: trending_date, dtype: datetime64[ns]

In [62]:
combined_df['year'] = combined_df['trending_date'].dt.year
combined_df['month'] = combined_df['trending_date'].dt.month
combined_df['day'] = combined_df['trending_date'].dt.day

In [63]:
combined_df[['trending_date', 'year', 'month', 'day']].head(10)

,trending_date,year,month,day
0,2017-11-14,2017,11,14
1,2017-11-14,2017,11,14
2,2017-11-14,2017,11,14
3,2017-11-14,2017,11,14
4,2017-11-14,2017,11,14
5,2017-11-14,2017,11,14
6,2017-11-14,2017,11,14
7,2017-11-14,2017,11,14
8,2017-11-14,2017,11,14
9,2017-11-14,2017,11,14


In [64]:
##Question13: For each (month, country) pair, the video with the largest number of views

In [65]:
idx_month_country = combined_df.groupby(
    ['month', 'country']
)['views'].idxmax()

In [66]:
idx_month_country.head()

month  country
1      CA         183337
       DE          88233
       FR         264589
       GB         347406
       IN          52208
Name: views, dtype: int64

In [67]:
top_month_country = combined_df.loc[idx_month_country].reset_index(drop=True)
top_month_country.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio,publish_time_10min,year,month,day
0,LsoLEjrDogU,2018-01-09,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars|""Finesse""|""Cardi B""|""Finesse Remix""...",43067983,1717177,61567,...,False,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,CA,27.891192,2018-01-04 04:40:00+00:00,2018,1,9
1,LsoLEjrDogU,2018-01-08,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars|""Finesse""|""Cardi B""|""Finesse Remix""...",37728802,1629946,56305,...,False,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,DE,28.948513,2018-01-04 04:40:00+00:00,2018,1,8
2,LsoLEjrDogU,2018-01-08,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars""|""Finesse""|""Cardi B""|""Finesse Remix...",37728802,1629948,56305,...,False,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,FR,28.948548,2018-01-04 04:40:00+00:00,2018,1,8
3,LsoLEjrDogU,2018-01-18,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,Bruno Mars,10,2018-01-04 04:49:43+00:00,"Bruno Mars|""Finesse""|""Cardi B""|""Finesse Remix""...",90598955,2248693,93089,...,False,False,False,Finesse (Remix) Feat. Cardi B Available Now: h...,GB,24.156377,2018-01-04 04:40:00+00:00,2018,1,18
4,dfnCAmr569k,2018-01-18,"Taylor Swift - End Game ft. Ed Sheeran, Future",TaylorSwiftVEVO,10,2018-01-12 05:00:01+00:00,"Taylor|""Swift""|""End""|""Game""|""Big""|""Machine""|""Pop""",42019590,1804377,100033,...,False,False,False,Music video by Taylor Swift performing End Gam...,IN,18.037818,2018-01-12 05:00:00+00:00,2018,1,18


In [68]:
##Question14: Read all json files with the video categories

In [69]:
import pandas as pd
import glob  
import json  
import os

In [70]:
json_files = glob.glob('*_category_id.json')
json_files

['IN_category_id.json',
 'RU_category_id.json',
 'JP_category_id.json',
 'FR_category_id.json',
 'GB_category_id.json',
 'KR_category_id.json',
 'MX_category_id.json',
 'CA_category_id.json',
 'US_category_id.json',
 'DE_category_id.json']

In [72]:
category_dfs = []

for file in json_files:
    country_code = os.path.basename(file)[:2] 
    
    with open(file, 'r', encoding='utf-8') as f:  
        data = json.load(f)  
    
    df_cat = pd.json_normalize(data['items'])   
    
    df_cat['country'] = country_code      
    
    category_dfs.append(df_cat)  

categories_df = pd.concat(category_dfs, ignore_index=True) 
categories_df.head()


,kind,etag,id,snippet.channelId,snippet.title,snippet.assignable,country
0,youtube#videoCategory,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True,IN
1,youtube#videoCategory,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True,IN
2,youtube#videoCategory,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True,IN
3,youtube#videoCategory,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True,IN
4,youtube#videoCategory,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True,IN


In [73]:
##Question15: For each country, determine how many videos have a category that is not assignable.

In [74]:
categories_df = categories_df.rename(columns={
    'id': 'category_id',
    'snippet.title': 'category_title',
    'snippet.assignable': 'assignable'
})


categories_df['category_id'] = categories_df['category_id'].astype(int)

categories_df[['country', 'category_id', 'category_title', 'assignable']].head()

,country,category_id,category_title,assignable
0,IN,1,Film & Animation,True
1,IN,2,Autos & Vehicles,True
2,IN,10,Music,True
3,IN,15,Pets & Animals,True
4,IN,17,Sports,True


In [118]:
videos_with_cat = combined_df.merge(
    categories_df[['country', 'category_id', 'assignable']],
    how='left',
    on=['country', 'category_id']
)
videos_with_cat[['country', 'category_id', 'assignable']].head()

,country,category_id,assignable
0,MX,24,True
1,MX,22,True
2,MX,25,True
3,MX,25,True
4,MX,26,True


In [76]:
not_assignable_videos = videos_with_cat[videos_with_cat['assignable'] == False]

not_assignable_counts = (
    not_assignable_videos
    .groupby('country')
    .size()
    .reset_index(name='n_videos_not_assignable')
)

not_assignable_counts

,country,n_videos_not_assignable
0,CA,130
1,DE,110
2,FR,112
3,GB,20
4,IN,221
5,KR,167
6,MX,3
7,RU,195
8,US,57
